<a href="https://colab.research.google.com/github/giuseppefutia/machine-learning/blob/master/pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pytorch Tutorial

In [0]:
import torch

## Some points on view() method

One of the firt basic operation that I want to perform on Pytorch is the use of view method of a tensor.

The view method reshapes a tensor, also when you don't know which dimension should have the row (or the column) of you reshaped matrix.

As you can see in the  following example, you can set the value -1, at the column place.


In [0]:
x = torch.randn(4,4)
print(x)
print(x.size())

print('---')

y = x.view(8, 2)
print(y)
print(y.size())

print('---')

z = x.view(8, -1)
print(z)
print(z.size())

tensor([[ 0.2670,  0.1353, -0.6542, -0.7428],
        [ 0.9956, -0.2225,  0.9738, -0.4819],
        [ 2.1576, -1.1777, -0.6376, -0.1671],
        [ 1.6130, -1.4044,  0.4690,  0.0457]])
torch.Size([4, 4])
---
tensor([[ 0.2670,  0.1353],
        [-0.6542, -0.7428],
        [ 0.9956, -0.2225],
        [ 0.9738, -0.4819],
        [ 2.1576, -1.1777],
        [-0.6376, -0.1671],
        [ 1.6130, -1.4044],
        [ 0.4690,  0.0457]])
torch.Size([8, 2])
---
tensor([[ 0.2670,  0.1353],
        [-0.6542, -0.7428],
        [ 0.9956, -0.2225],
        [ 0.9738, -0.4819],
        [ 2.1576, -1.1777],
        [-0.6376, -0.1671],
        [ 1.6130, -1.4044],
        [ 0.4690,  0.0457]])
torch.Size([8, 2])


## Cuda Tensors
A simple way to verify of your cuda architecture is available.

In [0]:
if torch.cuda.is_available():
  device = torch.device('cuda')
  print(device)
else:
  print('cuda not available')

cuda not available


## Autograd

The idea behind autograd is the following: when you compute an operation on a Tensor where the parameter requires_grad is set to True, a gradient function is associated to this operation. This gradient function will be employed during the backpropagation step.

In [0]:
x = torch.ones(2, 2, requires_grad=True)
print(x)
print()
print()

y = x+2
print(y)
print()
print()

z = y * y * 3
print(z)
print()
print()


out = z.mean()
print(out)



tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>)


tensor(27., grad_fn=<MeanBackward1>)


## Gradients
You can perform the backpropagation using the following command:

In [0]:
out.backward()

Print gradients d(out)/dx

In [0]:
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


## *** TODO NEED TO SEE IN A BETTER WAY SOMETHING RELATED TO THE JACOBIAN MATRIX

## Neural Networks

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F

def __init__(self):
        super(Net, self).__init__()
        
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
        

## Two-layered Network
In this first example, we see how to train a shallow neural network with numpy and pytorch.

This example is written starting from the code published at the following link: https://pytorch.org/tutorials/beginner/pytorch_with_examples.html

#### Tensors

In this example we adopt pytorch tensors for input, output data, and weights.

Here, we manually perform forward and backward (as we made in the previous step with numpy).

In [0]:
import torch


dtype = torch.float
device = torch.device("cpu")

if torch.cuda.is_available(): # Try to use GPU if available
  device = torch.device('cuda')

print(device)

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random input and output data
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# Randomly initialize weights
w1 = torch.randn(D_in, H, device=device, dtype=dtype)
w2 = torch.randn(H, D_out, device=device, dtype=dtype)

# Define the learning rate
learning_rate = 1e-6

for t in range(500):
  
    # Forward pass: compute predicted y
    # mm() performs a matrix multiplication of x and w1
    h = x.mm(w1)
    # clam() allows you to construct the relu. The output will be the max betwwen 0 and the number
    h_relu = h.clamp(min=0)
    y_pred = h_relu.mm(w2)

    # Compute and print loss
    # Using item, I can get the effective value of loss (otherwise I will get a tensor)
    loss = (y_pred - y).pow(2).sum().item()
    
    if (t % 100) == 0:
      print('epoch:' + str(t), 'loss:' + str(loss))

    # Backprop to compute gradients of w1 and w2 with respect to loss
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.t().mm(grad_y_pred)
    grad_h_relu = grad_y_pred.mm(w2.t())
    grad_h = grad_h_relu.clone()
    grad_h[h < 0] = 0
    grad_w1 = x.t().mm(grad_h)

    # Update weights using gradient descent
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

#### Autograd
In this example we do not manually perform forward and backward propagation, but we exploit the autograd feature of Pytorch in order to automate both the steps.

With the autograd function, a computational graph will be built during the forward pass. For each computation a derivative will be generated, that will be used during the backprop.

In [0]:
import torch

dtype = torch.float
device = torch.device("cpu")
if torch.cuda.is_available(): # Try to use GPU if available
  device = torch.device('cuda')

print(device)

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold input and outputs.
# Setting requires_grad=False indicates that we do not need to compute gradients
# with respect to these Tensors during the backward pass.
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# Create random Tensors for weights.
# Setting requires_grad=True indicates that we want to compute gradients with
# respect to these Tensors during the backward pass.
w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

# Define the learning rate
learning_rate = 1e-6

for t in range(500):
    # Forward pass: compute predicted y using operations on Tensors; these
    # are exactly the same operations we used to compute the forward pass using
    # Tensors, but we do not need to keep references to intermediate values since
    # we are not implementing the backward pass by hand.
    y_pred = x.mm(w1).clamp(min=0).mm(w2)

    # Compute and print loss using operations on Tensors.
    # Now loss is a Tensor of shape (1,)
    # loss.item() gets the a scalar value held in the loss.
    loss = (y_pred - y).pow(2).sum()
    if (t % 100) == 0:
      print('epoch:' + str(t), 'loss:' + str(loss.item()))

    # Use autograd to compute the backward pass. This call will compute the
    # gradient of loss with respect to all Tensors with requires_grad=True.
    # After this call w1.grad and w2.grad will be Tensors holding the gradient
    # of the loss with respect to w1 and w2 respectively.
    loss.backward()

    # Manually update weights using gradient descent. Wrap in torch.no_grad()
    # because weights have requires_grad=True, but we don't need to track this
    # in autograd.
    # An alternative way is to operate on weight.data and weight.grad.data.
    # Recall that tensor.data gives a tensor that shares the storage with
    # tensor, but doesn't track history.
    # You can also use torch.optim.SGD to achieve this.
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad

        # Manually zero the gradients after updating weights
        w1.grad.zero_()
        w2.grad.zero_()

#### nn

Computational graphs and autograd are very powerful to define complex operators and automatically compute derivatives.

Traditionally, when we build neural networks, we usually arrange computation in layers that contain learnable parameters which will be optimized during the learning stage.

In [0]:
import torch

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold inputs and outputs
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# Use the nn package to define our model as a sequence of layers. nn.Sequential
# is a Module which contains other Modules, and applies them in sequence to
# produce its output. Each Linear Module computes output from input using a
# linear function, and holds internal Tensors for its weight and bias.
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)

# The nn package also contains definitions of popular loss functions; in this
# case we will use Mean Squared Error (MSE) as our loss function.
loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-4

for t in range(500):
    # Forward pass: compute predicted y by passing x to the model. Module objects
    # override the __call__ operator so you can call them like functions. When
    # doing so you pass a Tensor of input data to the Module and it produces
    # a Tensor of output data.
    y_pred = model(x)

    # Compute and print loss. We pass Tensors containing the predicted and true
    # values of y, and the loss function returns a Tensor containing the
    # loss.
    loss = loss_fn(y_pred, y)
    if (t % 100) == 0:
      print('epoch:' + str(t), 'loss:' + str(loss.item()))

    # Zero the gradients before running the backward pass.
    # We need to set the gradients to zero before the backward pass, because
    # Pytorch accumulates the gradients on subsequent backward passes. The risk
    # is that gradients can point in some other directions than the minimum.
    model.zero_grad()

    # Backward pass: compute gradient of the loss with respect to all the learnable
    # parameters of the model. Internally, the parameters of each Module are stored
    # in Tensors with requires_grad=True, so this call will compute gradients for
    # all learnable parameters in the model.
    loss.backward()

    # Update the weights using gradient descent. Each parameter is a Tensor, so
    # we can access its gradients like we did before.
    with torch.no_grad():
      for param in model.parameters():
        param -= learning_rate * param.grad

#### Optim

Until now, the parameters have been manually updated, avoiding tracking history in autograd (using for instance *torch.no_grad()*) .

Nevertheless, if we want to adopt more sophisticated optimizers, we can use the optim package of pytorch as follows.

In [0]:
# -*- coding: utf-8 -*-
import torch

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold inputs and outputs
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# Use the nn package to define our model and loss function.
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)
loss_fn = torch.nn.MSELoss(reduction='sum')

# Use the optim package to define an Optimizer that will update the weights of
# the model for us. Here we will use Adam; the optim package contains many other
# optimization algoriths. The first argument to the Adam constructor tells the
# optimizer which Tensors it should update.
learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
for t in range(500):
    # Forward pass: compute predicted y by passing x to the model.
    y_pred = model(x)

    # Compute and print loss.
    loss = loss_fn(y_pred, y)
    if (t % 100) == 0:
      print('epoch:' + str(t), 'loss:' + str(loss.item()))

    # Before the backward pass, use the optimizer object to zero all of the
    # gradients for the variables it will update (which are the learnable
    # weights of the model). This is because by default, gradients are
    # accumulated in buffers( i.e, not overwritten) whenever .backward()
    # is called. Checkout docs of torch.autograd.backward for more details.
    optimizer.zero_grad()

    # Backward pass: compute gradient of the loss with respect to model
    # parameters
    loss.backward()

    # Calling the step function on an Optimizer makes an update to its
    # parameters
    optimizer.step()